In [ ]:
# Notes to self:

# The clustering on skills didn't work because there isn't enough skills per user.
# Almost all users have about 5 skills, but there isn't enough overlap between them.
# Futhermore, there is just about no signal relating the features to hourly rate.
# Need to pivot and think of some alternative way to use this data.
# Also, I need to figure out why everyone has a "1" for their invoices paid.

### Importing and Merging to get EDA Dataset

In [5]:
# Packages for PostgreSQL Import and Export
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import create_engine
import psycopg2

# Packages for EDA
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from datetime import datetime
from nltk.corpus import stopwords

# Packages for K-Modes Cluster
import numpy as np
from kmodes.kmodes import KModes

In [6]:
# Ideally I'll move this into the project config.py file
# Otherwise for now I have to just manually assign . . .

dbname = "freelance_db"
username = "Metaverse"
pswd = "Arcifice91"

# Connect to Data
con = None
con = psycopg2.connect(database=dbname, user=username,
                       host='localhost', password=pswd)

# Checking shapes of tables
sql_query = """SELECT * from user_details_table;"""
dtls_table = pd.read_sql_query(sql_query, con)

sql_query = """SELECT * from freelance_table;"""
fl_table = pd.read_sql_query(sql_query, con)

In [7]:
# Merging
# Modifying profile_url in the details table so I can merge it with the fl_table
dtls_table['profile_url'] = "https://www.guru.com"+dtls_table['profile_url']

# Now I'm merging them together to make dt
dt = pd.merge(fl_table, dtls_table, on='profile_url')
dt.shape
dt = dt.drop(columns=(['index_x']))
dt = dt.drop_duplicates(subset='profile_url')

In [8]:
# Merging in data to crosswalk states with region labels
state_region_crswlk = pd.read_csv(
    '~/Desktop/Insight/projects/myrate/data/raw/census-regions/us census bureau regions and divisions.csv')
state_region_crswlk.head()

dt = pd.merge(dt, state_region_crswlk,
                     how='left', left_on='state', right_on='State')

# Fixing region value for individuals living in Puerto Rico
dt.loc[dt['state'] == "Puerto Rico", 'Region'] = "Other"

# Exploratory Data Analysis - Outcome: Hourly Rate
### Beginning with investigating the geographic labels 

In [50]:
# What is the overall mean?
print("Mean hourly rate ($):",round(dt['hourly_rate'].mean(),2))

Mean hourly rate ($): 31.94


In [51]:
# What is the mean by region?
dt.groupby(['Region']).hourly_rate.mean()

# There is some variation, but it is minimal

Region
Midwest      29.724138
Northeast    30.782609
Other        20.000000
South        31.972222
West         34.197719
Name: hourly_rate, dtype: float64

In [52]:
# What is the mean across states?
dt.groupby(['state']).hourly_rate.mean().sort_values().describe()

# Pretty decent spread. This may be the most powerful signal yet.
# If you print out hte full table it is clear that there is variation

count     50.000000
mean      33.764035
std       15.197060
min        8.000000
25%       26.650000
50%       32.175057
75%       36.732353
max      104.000000
Name: hourly_rate, dtype: float64

In [53]:
# What is the variation within states?
state_lvl_hourly_rate_descriptives = {}
state_lvl_hourly_rate_descriptives['min'] = dt.groupby(['state']).hourly_rate.min()
state_lvl_hourly_rate_descriptives['mean'] = dt.groupby(['state']).hourly_rate.mean()
state_lvl_hourly_rate_descriptives['max'] = dt.groupby(['state']).hourly_rate.max()
state_lvl_hourly_rate_descriptives['var'] = dt.groupby(['state']).hourly_rate.var()
dt_state_hr = pd.DataFrame(state_lvl_hourly_rate_descriptives)

# A decent amount of variation within the states

In [54]:
# I tried to break it down further to the city level
# But the city data is really messy and not worth the time sink
# To try and get it set up. The idea would be to merge on
# rural vs urban and see if it can help me filter out a signal.

### Investigating the relationship between a users "rating" and the outcome

In [55]:
# Cleaing it up real quick
dt.rating = dt.rating.str.replace('%', '')
dt.rating = dt.rating.str.replace('NA', '')
dt.rating = pd.to_numeric(dt.rating)

In [60]:
# Starting with Missing Values
print("Number of missing values: ",dt.rating.isna().sum())

Number of missing values:  68


In [75]:
# What does the spread look like for the rest?
dt.rating.describe()

# Basically no variation. Okay, let's make a binary variable (has_rating)

count    911.000000
mean      98.424040
std       11.376479
min        1.000000
25%      100.000000
50%      100.000000
75%      100.000000
max      100.000000
Name: rating, dtype: float64

In [78]:
# Creating "has_rating"
dt['has_rating'] = None
dt.loc[dt['rating'].isna(), 'has_rating'] = 0
dt.loc[dt['rating'] > 0, 'has_rating'] = 1

# Comparing outcome means by has_rating
dt.groupby(['has_rating']).hourly_rate.mean()

# Small difference, nothing substantial

has_rating
0    34.529412
1    31.743139
Name: hourly_rate, dtype: float64

### Exploring the duration of membership and hourly rate

1. Convert member start date to a date object
2. Calculate years / months they've been a member (to today)

In [79]:
def date_convert(member_since):
    try:
        tmp = datetime.strptime(member_since, '%b, %Y')
        #tmp = tmp.strftime('%Y-%m')
    except:
        tmp = 'NaN'
    return tmp

In [80]:
def years_active(date):
    cur_year = datetime.now().year
    try:
        yrs_active = cur_year - date.year
    except:
        yrs_active = 'NaN'

    return yrs_active

In [81]:
def months_active(date):
    cur_year = datetime.now().year
    cur_month = datetime.now().month

    try:
        mnths_active = (cur_year - date.year)*12 + (cur_month - date.month)
    except:
        mnths_active = 'NaN'

    return mnths_active

In [83]:
# Converting member_since to date
dt.member_since = dt.member_since.str.strip()
dt['start_date'] = dt.member_since.apply(date_convert)
dt['years_active'] = dt.start_date.apply(years_active)
dt['months_active'] = dt.start_date.apply(months_active)

# Therea are 31 NAs. Just going to ignore them for now.

In [95]:
# What I want to understand is:
# How does the hourly rate (average) change across years active?
dt.groupby('years_active').hourly_rate.mean()

# Nothing much. It ramps up if years active > 17. Building that binary.

In [96]:
dt['active_17up'] = (dt['years_active'] >= 17) * 1

In [ ]:
# Exploring distribution of years active
sns.distplot(dt.years_active)

In [ ]:
# Exploring distribution of months active
sns.distplot(dt.months_active)

In [ ]:
# Discussing distributions of years active
# Pretty similar, heavy towards the low end with some people
# who have been active for a very long time.

In [ ]:
# How does it compare with hourly rate? Not much there.
sns.scatterplot(x='hourly_rate', y='years_active', data=dt)

In [ ]:
sns.scatterplot(x='hourly_rate', y='months_active', data=dt)

In [ ]:
# What about by region? I don't think there would be much here. Not worth the investment.

### Moving on to invoices rate

In [ ]:
# What about the (obvious) relationship between memebershp time and # of jobs completed
# How strong is the relationship?
# First have to clean the invoices paid variable
dt.invoices_paid = dt.invoices_paid.str.replace(',', '')
dt.invoices_paid = pd.to_numeric(dt.invoices_paid)
dt.invoices_paid.describe()

In [ ]:
# Who is NA? I have a whole bunch of 1s but then some NAs?
# The question for a poisson model is: did they just have no invoices OR
# were they never going to have any invoices.
# Looked at some of them and I'll treat them as zeros. So there are really that many ones?!?
dt[dt.invoices_paid.isna()].head()

In [ ]:
# Assigning zero to the NAs. This is important!!
dt.loc[dt.invoices_paid.isna(), 'invoices_paid'] = 0

In [ ]:
sns.distplot(dt.invoices_paid)

In [ ]:
# Wow, very heavy on the zeros. Let's look closer.
dt.invoices_paid.describe()

In [ ]:
# Ah, so it is truncating at one.
# This is going to be problematic for those who are actually at a value of one.
# I can either rerun the scrape and create something to make the val = 0
# or I can just move forward. . .

# For now I'm going to keep moving forward and treat 1 as 0.
# Note, it is more likely than not that they are = 0

In [ ]:
# Checking dist vs months active (no 1s)
sns.scatterplot(x='months_active', y='invoices_paid',
                data=dt[dt.invoices_paid > 1])

In [ ]:
# A few massive outliers. Going to rough chop it down and replot
invoice_gtr_one = dt.invoices_paid > 1
invoice_ls_2000 = dt.invoices_paid < 2000
sns.scatterplot(x='months_active', y='invoices_paid',
                data=dt[invoice_gtr_one & invoice_ls_2000])

In [ ]:
# hmm, there may be a slight signal but you would expect a stronger trend

In [ ]:
# Checking dist vs years active (no 1s)
sns.scatterplot(x='years_active', y='invoices_paid',
                data=dt[invoice_gtr_one & invoice_ls_2000])
# Slight postive non-linear trend bu there is a HUGE outlier

### Invoices Paid - Notes

It has a weak-ish relationship with time. There are a lot of really heavy users of the platform that dominate the market. Let's zoom into the first year and see what it looks like.

In [ ]:
# Looking at invoices pad in the first few months (among those only active for a shorter period of time)
sns.scatterplot(x='months_active',
                y='invoices_paid',
                data=dt[dt['years_active'] < 2])

In [ ]:
# Calculating invoices / time active
dt['invoices_per_month'] = dt['invoices_paid'] / \
    dt['months_active']
sns.distplot(dt.invoices_per_month)
print(dt.invoices_per_month.describe())

In [ ]:
# Loaded on zero with a long tail. Continues to sell that there are some serious power users.
# Average invoice per month is equal to one, but I left in the 1s that could be zeros. . .
# Real average is likely near zero.

# The question now is: Does this mean differ by any characteristic that I observe in the data?
# For people with a different skill set (Andriod) do they have a higher mean invoice/month rate?
# For people in different regions do they have a higher mean invoice/month rate?
# For people with lower hourly rates do they have a higher mean invoice/month rate??

# The next important step is to figure out how to use the skills data.
# It would be great to try and create clusters of the skills.

### Working with Skills Data

1. Obtain the dataset that is long by skills and shrink it down to just the skills. I'll work with that for this.
2. Create some quick features such as: # of skills and top skill. These are user level so I can attach them back to dt to have everything in one place.

Note: the skills are standardized on guru which makes this more straightforward

In [ ]:
sql_query = """SELECT profile_url, skills_list from freelance_table;"""
skills_table = pd.read_sql_query(sql_query, con)
skills_table_dummies = pd.get_dummies(skills_table, columns=[
                                      'skills_list'], prefix='').groupby(['profile_url']).sum()

In [ ]:
# Number of skills
num_skills = skills_table.groupby('profile_url').count()
num_skills.skills_list.describe()

# Relatively uninformative because everyone maxes out the number of skilsl they can have.
# However, I may be able to get some predictive power off the small group of people who have < 5 skills.

In [ ]:
# Flagging those with < 5 skills
num_skills['less_five_skills'] = (num_skills.skills_list < 5) * 1

In [ ]:
# Extracting First Skill
first_skill = skills_table[skills_table.groupby(
    'profile_url')['skills_list'].cumcount() == 0]
first_skill = first_skill.rename(
    columns={"profile_url": "profile_url", "skills_list": "first_skill"})
first_skill.head()

In [ ]:
num_skills = pd.merge(num_skills, first_skill, on='profile_url')
num_skills.head()

In [ ]:
# Merging into dt
dt = pd.merge(dt, num_skills, on="profile_url")

### Working on clustering the skills

In [ ]:
# Found K-Modes on stackoverflow
# Following the package documentation
# Ref: https://pypi.org/project/kmodes/
# Ref: https://stackoverflow.com/questions/42639824/python-k-modes-explanation
# Ref: https://www.kaggle.com/ashydv/bank-customer-clustering-k-modes-clustering

In [ ]:
# Cleaning column names from the dummies database
skills_table_dummies.columns = skills_table_dummies.columns.str.strip().str.lower().str.replace(
    ' ', '_').str.replace('(', '').str.replace(')', '').str.replace('_','')

In [ ]:
skills_table_dummies.head()

In [ ]:
# Modeling with K Modes
cost = []
for num_clusters in list(range(20,30)):
    kmode = KModes(n_clusters=num_clusters, init = "Huang", verbose=0)
    kmode.fit_predict(skills_table_dummies)
    cost.append(kmode.cost_)
    print("Finished Cluster: " + str(num_clusters))

In [ ]:
y = np.array([i for i in range(20,30,1)])
plt.plot(y,cost)

In [ ]:
kmode = KModes(n_clusters=28, init='Huang', verbose=0)
clusters = kmode.fit_predict(skills_table_dummies)

kmodes = kmode.cluster_centroids_
shape = kmodes.shape

for i in range(shape[0]):
    if sum(kmodes[i,:]) == 0:
        print("\ncluster " + str(i) + ": ")
        print("no-skills cluster")
    else:
        print("\ncluster " + str(i) + ": ")
        cent = kmodes[i,:]
        for j in skills_table_dummies.columns[np.nonzero(cent)]:
            print(j)

In [ ]:
clust_assigned = km.predict(skills_table_dummies)

In [ ]:
unique, counts = np.unique(clust_assigned, return_counts=True)
dict(zip(unique, counts))

In [ ]:
user_cluster_crosswalk = pd.DataFrame(
    skills_table['profile_url'].unique(), columns=["profile_url"]).sort_values(by="profile_url")
user_cluster_crosswalk['cluster'] = clust_assigned
user_cluster_crosswalk[user_cluster_crosswalk['cluster'] == 0].head()

In [ ]:
# Merging into the skills table to investigate those with "no skills"
tmp = pd.merge(skills_table, user_cluster_crosswalk, on = "profile_url")
tmp[tmp['cluster'] == 0]

In [ ]:
# It seems to not be working great. 
# Why is it putting so many people into the no skills cluster?

### Starting the process of using the "bio" data

1. The easiest variable to create is the len(bio)
2. I'll have to try and expand it more using some other techniques

In [ ]:
def clean_bio(bio):
    try:
        cleaned_bio = ''.join(s for s in bio if ord(s) > 31 and ord(s) < 126)
    except:
        cleaned_bio = "NaN"
    return cleaned_bio

In [ ]:
def len_bio(bio):
    if bio == "NaN":
        return 0
    else:
        return len(bio)

In [ ]:
dt['bio_clean'] = dt.bio.apply(clean_bio)

In [ ]:
dt['bio_length'] = dt.bio_clean.apply(len_bio)

In [ ]:
# Looking at the distrbution of bio lengths
sns.distplot(dt.bio_length)

# Nothing crazy, lots of 0s but a good number of people with bios.

In [ ]:
# How does it relate to invoices / month?
sns.scatterplot(x='bio_length',
                y='invoices_per_month',
                data=dt)

# It's weak but I'll keep it for now

In [ ]:
# Number of separate words in bio
dt['bio_word_count'] = dt['bio_clean'].apply(
    lambda x: len(str(x).split(" ")))

In [ ]:
# Average word length in bio
def avg_word_ln(bio):
    try:
        words = bio.split()
        res = (sum(len(word) for word in words)/len(words))
    except:
        res = 0
    return res

In [ ]:
dt['avg_word_length'] = dt['bio_clean'].apply(avg_word_ln)

In [ ]:
# Number of stop words
def num_stopwords(bio):
    stop = stopwords.words('english')

    try:
        res = len([x for x in bio.split() if x in stop])
    except:
        res = -1

    return res

In [ ]:
dt['num_stop'] = dt['bio_clean'].apply(num_stopwords)

In [ ]:
# Preprocessing before pulling more features

# Remvoing stop words
stop = stopwords.words('english')
dt['bio_processed'] = dt['bio_clean'].apply(
    lambda x: " ".join(x for x in x.split() if x not in stop))

# Removing punctuation
dt['bio_processed'] = dt['bio_processed'].str.replace(
    '[^\w\s]', '')

# Lower Case
dt['bio_processed'] = dt['bio_processed'].apply(
    lambda x: " ".join(x.lower() for x in x.split()))

### Some final cleaning

In [ ]:
# Making Region into dummy variables
region_dummies = pd.get_dummies(dt['Region'])
dt = pd.concat([dt, region_dummies], axis=1)

In [ ]:
# Earnings per Month
dt['earnings_ever'] = dt['earnings_ever'].str.replace('$', '')
dt['earnings_ever'] = dt['earnings_ever'].str.replace(',', '')
dt['earnings_ever'] = pd.to_numeric(dt['earnings_ever'])
dt['earnings_pr_month'] = dt['earnings_ever'] / \
    dt['months_active']

### Saving out Modified Dataset

1. First printing the columns I have for reference.
2. Saving it to a new table in the postgreSQL database

In [ ]:
dt.info()

In [ ]:
# Cleaning to just variables that will be used in the model
analysis_dt = dt[['invoices_per_month', 'hourly_rate', 'earnings_pst_yr', 'earnings_pr_month',
                         'Midwest', 'Northeast', 'South', 'Other', 'West', 'less_five_skills', 'bio_length',
                         'bio_word_count', 'avg_word_length', 'num_stop', 'bio_processed']]
analysis_dt = analysis_dt.dropna()  # Removing people with NA
analysis_dt.info()

In [ ]:
# Connect to the database and save data to it
engine = create_engine('postgresql://%s:%s@localhost/%s' %
                       (username, pswd, dbname))

In [ ]:
analysis_dt.to_sql("analysis_table", engine, if_exists='replace')